In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re, pip, conda

In [ ]:
#pip install --upgrade scikit-learn
#conda update scikit-learn

# 二 贝叶斯优化的实现

贝叶斯优化是当今黑盒函数估计领域最为先进和经典的方法，在同一套序贯模型下使用不同的代理模型以及采集函数、还可以发展出更多更先进的贝叶斯优化改进版算法，因此，贝叶斯优化的其算法本身就多如繁星，实现各种不同种类的贝叶斯优化的库也是琳琅满目，几乎任意一个专业用于超参数优化的工具库都会包含贝叶斯优化的内容。我们可以在以下页面找到大量可以实现贝叶斯优化方法的HPO库：https://www.automl.org/automl/hpo-packages/ ，其中大部分库都是由独立团队开发和维护，因此不同的库之间之间的优劣、性格、功能都有很大的差异。在课程中，我们将介绍如下三个可以实现贝叶斯优化的库：`bayesian-optimization`，`hyperopt`，`optuna`。

|HPO库|优劣评价|推荐指数|
|-|-|-|
|**bayes_opt**|✅实现基于高斯过程的贝叶斯优化<br>✅当参数空间由大量连续型参数构成时<br><br>⛔包含大量离散型参数时避免使用<br>⛔算力/时间稀缺时避免使用|⭐⭐|
|**hyperopt**|✅实现基于TPE的贝叶斯优化<br>✅支持各类提效工具<br>✅进度条清晰，展示美观，较少怪异警告或报错<br>✅可推广/拓展至深度学习领域<br><br>⛔不支持基于高斯过程的贝叶斯优化<br>⛔代码限制多、较为复杂，灵活性较差|⭐⭐⭐⭐|
|**optuna**|✅（可能需结合其他库）实现基于各类算法的贝叶斯优化<br>✅代码最简洁，同时具备一定的灵活性<br>✅可推广/拓展至深度学习领域<br><br>⛔非关键性功能维护不佳，有怪异警告与报错|⭐⭐⭐⭐|

注意，以上三个库<font color="red">**都不支持基于Python环境的并行或加速**</font>，大多数优化算法库只能够支持基于数据库（如MangoDB，mySQL）的并行或加速，但以上库都可以被部署在分布式计算平台。

三个库极其辅助包的安装方法分别如下，使用pip或conda安装时注意关闭梯子。

- Bayes_opt

In [ ]:
#!pip install bayesian-optimization
#!conda install -c conda-forge bayesian-optimization

- Hyperopt

In [ ]:
#!pip install hyperopt

- Optuna

In [ ]:
#!pip install optuna
#!conda install -c conda-forge optuna

- Skopt（作为Optuna辅助包安装，也可单独使用）

In [ ]:
#!pip install scikit-optimize

接下来我们会分别使用三个库来实现贝叶斯优化。在课程中，我们依然使用集成算法中的房价数据作为验证数据，并且呈现出我们之前在不同优化方法上得出的结果作为对比。同时，我们将使用与集成算法中完全一致的随机数种子、以及随机森林算法作为被优化的评估器。

- **导入库，确认使用数据**

In [4]:
#基本工具
import numpy as np
import pandas as pd
import time
import os #修改环境设置

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold, cross_validate

#优化器
from bayes_opt import BayesianOptimization

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

import optuna

Bayes_opt版本：1.2.0

In [5]:
print(optuna.__version__)

2.10.0


In [6]:
print(hyperopt.__version__)

0.2.7


In [7]:
data = pd.read_csv(r"D:\myJupyter\机器学习\datasets\House Price\train_encode.csv",index_col=0)

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [8]:
X.head()

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,半开放式门廊面积,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0


In [9]:
X.shape

(1460, 80)

- **确认该数据集上的历史成果**

|HPO方法|默认参数|网格搜索|随机搜索|随机搜索<br>(大空间)|随机搜索<br>(连续型)|
|:-:|:-:|:-:|:-:|:-:|:-:|
|搜索空间/全域空间|-|1536/1536|800/1536|1536/3000|1536/无限|
|运行时间（分钟）|-|6.36|<font color="green">**2.83(↓)**</font>|<font color="green">**3.86(↓)**</font>|3.92|
|搜索最优（RMSE）|30571.266|29179.698|29251.284|<font color="green">**29012.905(↓)**</font>|29148.381|
|重建最优（RMSE）|-|28572.070|<font color="brown">**28639.969(↑)**</font>|<font color="green">**28346.673(↓)**</font>|28495.682|

## 1 基于Bayes_opt实现GP优化

bayes-optimization是最早开源的贝叶斯优化库之一，也是为数不多至今依然保留着高斯过程优化的优化库。由于开源较早、代码简单，bayes-opt常常出现在论文、竞赛kernels或网络学习材料当中，因此理解Bayes_opt的代码是极其重要的课题。不过，bayes-opt对参数空间的处理方式较为原始，也缺乏相应的提效/监控功能，对算力的要求较高，因此它往往不是我们进行优化时的第一首选库。通常来说，当且仅当我们必须要实现基于高斯过程的贝叶斯优化，且算法的参数空间中带有大量连续型参数时，我们才会优先考虑Bayes_opt库。我们可以在github上找到bayes-optmization的官方文档（https://github.com/fmfn/BayesianOptimization） ，想要进一步了解其基本功能与原理的小伙伴可以进行阅读。

In [39]:
from bayes_opt import BayesianOptimization

- 1 定义目标函数

目标函数的值即$f(x)$的值。贝叶斯优化会计算$f(x)$在不同$x$上的观测值，因此$f(x)$的计算方式需要被明确。在HPO过程中，我们希望能够筛选出令模型泛化能力最大的参数组合，因此$f(x)$应该是损失函数的交叉验证值或者某种评估指标的交叉验证值。**需要注意的是，bayes_opt库存在三个影响目标函数定义的规则**：

> **1 目标函数的输入必须是具体的超参数，而不能是整个超参数空间，更不能是数据、算法等超参数以外的元素**，因此在定义目标函数时，我们需要让超参数作为目标函数的输入。<br><br>
> **2 超参数的输入值只能是浮点数，不支持整数与字符串**。因此当算法的实际参数需要输入字符串时，该参数不能使用bayes_opt进行调整，当算法的实际参数需要输入整数时，则需要在目标函数中规定参数的类型。<br><br>
> **3 bayes_opt只支持寻找$f(x)$的最大值，不支持寻找最小值**。因此当我们定义的目标函数是某种损失时，目标函数的输出需要取负（即，如果使用RMSE，则应该让目标函数输出负RMSE，这样最大化负RMSE后，才是最小化真正的RMSE。）当我们定义的目标函数是准确率，或者auc等指标，则可以让目标函数的输出保持原样。

In [62]:
def bayesopt_objective(n_estimators,max_depth,max_features,min_impurity_decrease):
    
    #定义评估器
    #需要调整的超参数等于目标函数的输入，不需要调整的超参数则直接等于固定值
    #默认参数输入一定是浮点数，因此需要套上int函数处理成整数
    reg = RFR(n_estimators = int(n_estimators)
              ,max_depth = int(max_depth)
              ,max_features = int(max_features)
              ,min_impurity_decrease = min_impurity_decrease
              ,random_state=1412
              ,verbose=False #可自行决定是否开启森林建树的verbose
              ,n_jobs=-1)
    
    #定义损失的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    #注意，交叉验证需要使用数据，但我们不能让数据X,y成为目标函数的输入
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise'
                                     #如果交叉验证中的算法执行报错，则告诉我们错误的理由
                                    )
    
    #交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    #目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

- 2 定义参数空间

在任意超参数优化器中，优化器会将参数空格中的超参数组合作为备选组合，一组一组输入到算法中进行训练。在贝叶斯优化中，超参数组合会被输入我们定义好的目标函数$f(x)$中。

在bayes_opt中，我们使用字典方式来定义参数空间，其中参数的名称为键，参数的取值范围为值。且任意参数的取值范围为双向闭区间，以下方的空间为例，在n_estimators的取值中，80与100都可以被取到。

以下参数空间与我们在随机森林中获得最高分的随机搜索的范围高度相似。

In [41]:
param_grid_simple = {'n_estimators': (80,100)
                     , 'max_depth':(10,25)
                     , "max_features": (10,20)
                     , "min_impurity_decrease":(0,1)
                    }

需要注意的是，bayes_opt只支持填写参数空间的上界与下界，不支持填写步长等参数，且bayes_opt会将所有参数都当作连续型超参进行处理，**因此bayes_opt会直接取出闭区间中任意浮点数作为备选参数**。例如，取92.28作为n_estimators的值。

这也是为什么在目标函数中，我们需要对整数型超参的取值都套上int函数。假设优化器取出92.28作为n_estimators的值，实际传入随机森林算法的会是int(92.28) = 92，如此我们可以保证算法运行过程中不会因参数类型不符而报错。也因为bayes_opt的这个性质，**输入bayes_opt的参数空间天生会比其他贝叶斯优化库更大/更密，因此需要的迭代次数也更多**。

- 3 定义优化目标函数的具体流程

在有了目标函数与参数空间之后，我们就可以按bayes_opt的规则进行优化了。在任意贝叶斯优化算法的实践过程中，一定都有涉及到随机性的过程——例如，随机抽取点作为观测点，随机抽样部分观测点进行采集函数的计算等等。**在大部分优化库当中，这种随机性是无法控制的**，即便允许我们填写随机数种子，优化算法也不能固定下来。因此我们可以尝试填写随机数种子，但需要记住优化算法每次运行时一定都会不一样。

虽然，优化算法无法被复现，但是优化算法得出的最佳超参数的结果却是可以被复现的。只要优化完毕之后，可以从优化算法的实例化对象中取出最佳参数组合以及最佳分数，该最佳参数组合被输入到交叉验证中后，是一定可以复现其最佳分数的。如果没能复现最佳分数，则是交叉验证过程的随机数种子设置存在问题，或者优化算法的迭代流程存在问题。

In [46]:
def param_bayes_opt(init_points,n_iter):
    
    #定义优化器，先实例化优化器
    opt = BayesianOptimization(bayesopt_objective #需要优化的目标函数
                               ,param_grid_simple #备选参数空间
                               ,random_state=1412 #随机数种子，虽然无法控制住
                              )
    
    #使用优化器，记住bayes_opt只支持最大化
    opt.maximize(init_points = init_points #抽取多少个初始观测值
                 , n_iter=n_iter #一共观测/迭代多少次
                )
    
    #优化完成，取出最佳参数与最佳分数
    params_best = opt.max["params"]
    score_best = opt.max["target"]
    
    #打印最佳参数与最佳分数
    print("\n","\n","best params: ", params_best,
          "\n","\n","best cvscore: ", score_best)
    
    #返回最佳参数与最佳分数
    return params_best, score_best

- 4 定义验证函数（非必须）

优化后的结果是可以复现的，即我们可以对优化算法给出的最优参数进行再验证，其中验证函数与目标函数高度相似，输入参数或超参数空间、输出最终的损失函数结果。在使用sklearn中自带的优化算法时，由于优化算法自己会执行分割数据、交叉验证的步骤，因此优化算法得出的最优分数往往与我们自身验证的分数不同（因为交叉验证时的数据分割不同）。然而在贝叶斯优化过程中，目标函数中的交叉验证即数据分割都是我们自己规定的，**因此原则上来说，只要在目标函数中设置了随机数种子，贝叶斯优化给出的最佳分数一定与我们验证后的分数相同**，所以当你对优化过程的代码比较熟悉时，可以不用进行二次验证。

In [48]:
def bayes_opt_validation(params_best):
    
    reg = RFR(n_estimators = int(params_best["n_estimators"]) 
              ,max_depth = int(params_best["max_depth"])
              ,max_features = int(params_best["max_features"])
              ,min_impurity_decrease = params_best["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1)

    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                    )
    return np.mean(validation_loss["test_score"])

- 5 执行实际优化流程

In [49]:
start = time.time()
params_best, score_best = param_bayes_opt(20,280) #初始看20个观测值，后面迭代280次
print('It takes %s minutes' % ((time.time() - start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score: ",validation_score)

|   iter    |  target   | max_depth | max_fe... | min_im... | n_esti... |
-------------------------------------------------------------------------
|  1        | -2.948e+0 |  23.2     |  17.52    |  0.06379  |  88.79    |
|  2        | -2.909e+0 |  14.8     |  17.61    |  0.9214   |  97.58    |
|  3        | -2.9e+04  |  15.86    |  15.56    |  0.2661   |  87.98    |
|  4        | -2.887e+0 |  14.05    |  16.84    |  0.06744  |  89.72    |
|  5        | -2.887e+0 |  18.71    |  19.17    |  0.9315   |  83.7     |
|  6        | -2.895e+0 |  17.7     |  19.58    |  0.7127   |  89.18    |
|  7        | -2.968e+0 |  14.21    |  12.62    |  0.3381   |  91.51    |
|  8        | -2.91e+04 |  23.23    |  10.89    |  0.6078   |  95.06    |
|  9        | -2.891e+0 |  14.89    |  14.0     |  0.9487   |  80.16    |
|  10       | -2.958e+0 |  11.52    |  12.58    |  0.03276  |  92.56    |
|  11       | -2.91e+04 |  13.14    |  13.31    |  0.2563   |  98.24    |
|  12       | -2.95e+04 |  17.94    | 

|HPO方法|默认参数|网格搜索|随机搜索|随机搜索<br>(大空间)|随机搜索<br>(连续型)|贝叶斯优化<br>(基于GP)|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|搜索空间/全域空间|-|1536/1536|800/1536|1536/3000|1536/无限|300/无限|
|运行时间（分钟）|-|6.36|<font color="green">**2.83(↓)**</font>|<font color="green">**3.86(↓)**</font>|3.92|<font color="green">**2.11(↓)**|
|搜索最优（RMSE）|30571.266|29179.698|29251.284|<font color="green">**29012.905(↓)**</font>|29148.381|<font color="green">**28346.673(↓)**</font>|
|重建最优（RMSE）|-|28572.070|<font color="brown">**28639.969(↑)**</font>|<font color="green">**28346.673(↓)**</font>|28495.682|<font color="green">**28346.673(↓)**</font>|

- 原理上有优越性

可以看到，基于高斯过程的贝叶斯优化在2.11分钟内锁定了最佳分数28346.673，这是之前使用随机搜索时获得的最佳分数，很可能也是我们当前超参数空间上可以获得的最佳分数。贝叶斯优化作为从原理上高于网格优化的HPO方法，能够以更短的时间获得与随机网格搜索相同的结果，可见其原理上的优越性。

- 优化过程无法复现，但优化结果可以复现

但同时要注意，由于贝叶斯优化每次都是随机的，因此我们并不能在多次运行代码时复现出28346.673这个结果，事实上如果我们重复运行，也只有很小的概率可以再次找到这个最低值（这一点对于随机搜索来说也是类似的，如果不规定随机数种子，我们也无法复现最低值）。因此我们在执行贝叶斯优化时，往往会多运行几次观察模型找出的结果。同时，验证分数与目标函数最后输出的分数一模一样，可见最终输出的超参数组合的效力是可以复现的。

- 效率不足

不难发现，bayes_opt的速度虽然快，效率却不高。实际上在迭代到170次时，贝叶斯优化就已经找到了最小损失，但由于没有提前停止机制，模型还持续地迭代了130次才停下，如果bayes_opt支持提前停止机制，贝叶斯优化所需的实际迭代时间可能会更少。同时，由于Bayes_opt只能够在参数空间提取浮点数，bayes_opt在随机森林上的搜索效率是较低的，即便在10次不同的迭代中分别取到了[88.89, 88.23, 88.16, 88.59……]等值，在取整之后也只能够获得一个备选值88，但bayes_opt无法辨别这种区别，因此可能取出了众多无效的观测点。如果使用其他贝叶斯优化器，贝叶斯优化的效率将会更高。

- 支持灵活修改

虽然在我们的代码中没有体现，但bayes_opt是支持灵活修改采集函数与高斯过程中的种种参数的，具体可以参考这里：https://github.com/fmfn/BayesianOptimization/blob/master/examples/advanced-tour.ipynb